In [41]:
import pyspark
import pymongo_spark
from pyspark import mllib
from pyspark.mllib.clustering import KMeans
import pymongo
from pymongo import MongoClient
from pyspark import SparkConf, SparkContext
pymongo_spark.activate()
client = MongoClient('10.120.28.20', 27017)
database = client["production"]
collection = database["ebc"]

sc.stop()
conf = (SparkConf().set("spark.driver.maxResultSize", "2g"))
sc = pyspark.SparkContext(master="local[*]", conf=conf, appName="word2vec")


In [106]:
import numpy as np
jieba.set_dictionary('/home/cloudera/Desktop/text_mining/dict.txt')            
jieba.load_userdict("/home/cloudera/Desktop/text_mining/dict_keyw_new.txt")
jieba.load_userdict("/home/cloudera/Desktop/text_mining/dict_cbdic.txt")
mongo_rdd = sc.mongoPairRDD("mongodb://10.120.28.20:27017/production.ebc") # 所有ebc新聞
mongo_rdd.count()

Building prefix dict from /home/cloudera/Desktop/text_mining/dict.txt ...
Loading model from cache /tmp/jieba.u77727ee2cf0db5a92286247fca7f8515.cache
Loading model cost 0.855 seconds.
Prefix dict has been built succesfully.


407753

In [95]:
from pyspark.mllib.feature import Word2Vec

content_RDD = mongo_rdd.map(lambda x: x[1]["tfidf"]) # 撈tfidf來跑word2vec
print content_RDD.first()

word2vec = Word2Vec()
model = word2vec.fit(content_RDD)  # 訓練word2vec model


[u'\u6d77\u5c3c\u6839', u'\u5564\u9152', u'\u5c0f\u7ad9', u'\u7159\u71fb', u'\u5831\u6599', u'\u5fb7\u570b', u'\u5716\u8d6b', u'\u5e97\u54e1', u'\u5169\u7f50', u'\u5564\u9152\u676f', u'\u7d10\u502b\u5821', u'\u694a\u5148\u751f', u'\u4e09\u829d', u'\u5ba2\u4eba', u'\u8ce3\u7684', u'\u8001\u95c6', u'\u8ce3\u7d66', u'\u54e1\u5de5', u'\u6ecb\u5473', u'\u5e97\u5167', u'\u79c1\u5e95\u4e0b', u'\u4e00\u53e3', u'\u88e1\u982d', u'\u81a9\u4e86', u'\u4e0d\u6703\u5427', u'tucher', u'660ml', u'\u7cbe\u91c0\u5564\u9152', u'\u8c8d\u8c93\u63db\u592a\u5b50', u'\u6d77\u5c3c', u'330ml', u'\u4e0b\u53e3', u'\u5fb7\u570b\u8c6c\u8173', u'\u8b00\u53d6', u'\u7b2c\u4e00\u676f', u'\u65b0\u5317\u5e02', u'500ml', u'\u919c\u4e8b', u'\u4e00\u559d', u'\u4e00\u676f\u9152']


In [98]:
# Save and load model
model.save(sc, "file:/home/cloudera/Desktop/text_mining/Word2VecModel")
#sameModel = Word2VecModel.load(sc, "myModelPath")

In [103]:
import time
start = time.time()
synonyms = model.findSynonyms("蔡英文",50)  # 查詢詞, 前n相關
for word, cosine_distance in synonyms:
    print("{}: {}".format(word.encode("utf8"), cosine_distance))
print time.time()-start # 查詢時間

民進黨: 2.08618758281
陳建仁: 2.02719778087
總統參選人: 2.02380895472
國民黨: 2.00834902248
朱立倫: 1.99331960609
第三勢力: 1.98581445417
蘇嘉全: 1.95825142207
政見: 1.95003058908
謝長廷: 1.94266911947
宋楚瑜: 1.94099343872
游梓翔: 1.93983920212
洪秀柱: 1.93694053056
兩岸政策: 1.93666553653
綠營: 1.93535072519
五大戰區: 1.93337271148
鄭運鵬: 1.93283584241
勝選: 1.93011731268
徐欣瑩: 1.92908315075
時代力量: 1.92298256783
總統候選人: 1.92282685537
黨代表: 1.9209031837
副手: 1.91947313334
許信良: 1.9184489688
辯論會: 1.91729597465
選對會: 1.91667515281
一中同表: 1.91451995821
政黨: 1.9130984626
政黨票: 1.90509661807
臨全會: 1.90365134515
藍營: 1.90126523051
蘇貞昌: 1.90075789637
選舉: 1.89874262869
徐巧芯: 1.89675763996
民國黨: 1.89407932185
蘇巧慧: 1.89182363673
劉康彥: 1.89181357664
台獨黨綱: 1.89177604946
林俊憲: 1.8905716861
親民黨: 1.885026101
挺柱: 1.88023099589
主席蔡英文: 1.87979835138
拔樁: 1.87616990892
立委參選人: 1.86914869204
泛藍: 1.86731990432
大黨: 1.86709746712
在野: 1.86520888179
柱柱姐: 1.86410922253
辯論: 1.86378725704
執政: 1.86355746566
中常會: 1.86130851035
0.363887071609


In [104]:
start = time.time()
synonyms = model.findSynonyms("王建民",50)
for word, cosine_distance in synonyms:
    print("{}: {}".format(word.encode("utf8"), cosine_distance))
print time.time()-start

建仔: 2.1886394043
3a: 2.16216662636
上大聯盟: 2.11974355279
王維中: 2.06433398061
曹錦輝: 2.0346258662
紅人: 2.02379613834
白襪: 2.02065013112
小聯盟: 2.01602240558
松坂大輔: 2.01167754367
運動家: 1.99427996545
道奇隊: 1.99391980615
雙城: 1.99139645897
曾仁和: 1.98817319972
小聯盟合約: 1.98717161623
響尾蛇: 1.98211726629
達比修: 1.97407416729
藍鳥: 1.96975196133
尹錫珉: 1.96844585483
紅人隊: 1.96065314995
李振昌: 1.95979874168
亞席沙: 1.95644853809
球探: 1.95178797026
大聯盟: 1.95070599886
印地安人: 1.94338295053
自責分率: 1.92505630982
波拉斯: 1.92472672524
2a: 1.91786011882
道奇: 1.91517239495
鈴木一朗: 1.91232207634
小熊隊: 1.90855350001
馬林魚: 1.90854474785
遊騎兵: 1.90667465807
印地安人隊: 1.90394227204
右投: 1.90368049326
范迪特: 1.89433159563
費城人: 1.89286048087
皇家隊: 1.88915602255
強投: 1.8884714844
陳偉殷: 1.88845889023
水手: 1.88741487723
田中將大: 1.88561832751
索茲: 1.885386016
金鶯隊: 1.88490296323
和田毅: 1.87703822184
美職: 1.87595751317
紅襪: 1.87432487494
教士: 1.87230385685
開投: 1.86844749658
洋基: 1.86551722623
勇士隊: 1.86267830295
0.218965053558


In [129]:
start = time.time()
synonyms = model.findSynonyms("雲端",50)
for word, cosine_distance in synonyms:
    print("{}: {}".format(word.encode("utf8"), cosine_distance))
print time.time()-start

行動裝置: 1.95984537543
應用: 1.89094957747
硬體: 1.85925098726
物聯網: 1.81439852515
解決方案: 1.79899250653
iaas: 1.77543345444
康容: 1.76161901877
裝置: 1.75865236972
大數據: 1.74260832132
使用者: 1.74056484279
雲端服務: 1.71357281438
軟體: 1.71250997518
cillin: 1.70657513082
趨勢科技: 1.7008283956
通訊: 1.69998246324
建置: 1.69307179869
程式: 1.69288274049
雙智: 1.6898387613
安裝: 1.68620208985
應用程式: 1.6758085544
數位: 1.67264967795
平板電腦: 1.67163645444
介面: 1.6701080732
運算: 1.66876066998
行動支付: 1.64854198803
語音: 1.64203696649
azure: 1.63452592787
智能: 1.63342701384
數位生活: 1.63243960976
蔡恩全: 1.62726719941
普及: 1.61925007456
導入: 1.61893089422
saas: 1.61850692062
新報: 1.61745323068
智慧型: 1.61518858355
轉換器: 1.61422954857
智慧家庭: 1.61305212517
擴充: 1.60999722476
智慧: 1.60869171907
推播: 1.60799040403
開發者: 1.60781658209
led: 1.60480229231
智慧型手機: 1.60426481995
行動通訊: 1.60356163661
維修中心: 1.60232677289
tizen: 1.60228320667
opera: 1.60091151092
微軟: 1.59670524973
用戶: 1.59461906379
mozil: 1.59416444674
0.210816144943


In [117]:
# 讀檔測試
from pyspark.mllib.feature import Word2VecModel
start = time.time()
wor2M = Word2VecModel.load(sc,"file:/home/cloudera/Desktop/text_mining/Word2VecModel")
synonyms = wor2M.findSynonyms("牛肉",50)
for word, cosine_distance in synonyms:
    print("{}: {}".format(word.encode("utf8"), cosine_distance))
print time.time()-start

# 確定pyspark的word2vec讀檔有bug, spark1.6版後才修正

ValueError: too many values to unpack